### Basis: CSV Parser

In [ ]:
class CSVParser:
    """Class voor verwerken van CSV-bestanden."""
    
    delimiter = ","
    
    def __init__(self, header=True, encoding="utf8"):
        self.header = header
        self.encoding = encoding
    
    @staticmethod
    def try_numeric(value):
        """Try to cast to numeric value."""
        try:
            return int(value)
        except ValueError:
            pass
        
        try:
            return float(value)
        except ValueError:
            return value
                
        
    def read_data(self, data_path):
        """Lees data en retourneer regels."""
        try:
            with open(data_path, "r", encoding=self.encoding) as data_file:
                
                # Eerste regel bevat kolomnamen
                if self.header == True:
                    header_line = next(data_file)
                    self.header = self.parse_record(header_line)
                
                return data_file.readlines()
        
        except FileNotFoundError as error:
            raise(f"Kan bestand {path} niet vinden.")
        
        except PermissionError as error:
            raise(f"Onvoldoende rechten om bestand {path} te lezen.")

    def parse_record(self, record):
        """Splits record in velden."""
        return [
            self.try_numeric(value.strip())
            for value in record.split(self.delimiter)
        ]
            
    def parse_data(self, data):
        """Knip regels op in dicts."""
        
        parsed = []
        for record in data:
            record = self.parse_record(record)
            
            # Maak dict als er een header is
            if self.header:
                if len(self.header) != len(record):
                    raise ValueError(f"Aantal kolommen niet consistent.")
                record = {name: value for name, value in zip(self.header, record)}
            
            parsed.append(record)
            
        return parsed
        
    def parse(self, data_path):
        """Verwerk een tekstbestand."""
        data = self.read_data(data_path)
        data = self.parse_data(data)
        
        return data

In [ ]:
parser = CSVParser()
parser.parse("dummy_data/delimited_data.csv")

### TSV Parser

In [ ]:
# TSVParser gebaseerd op CSV parser.
# Merk op: CSVParser staat als parent tussen de haken.
class TSVParser(CSVParser):
    """Class voor verwerken van CSV-bestanden."""
    
    # Alleen scheidingsteken hoeft aangepast te worden
    delimiter = "\t"

In [ ]:
parser = TSVParser()
parser.parse("dummy_data/delimited_data.tsv")

## Vaste kolommen

In [ ]:
# Merk op: FixedParser is gebaseerd op TextParser
class FixedParser(CSVParser):
    """Verwerkt bestanden met vaste kolommen."""
    
    # Aangepaste constructor
    def __init__(self, column_widths, header=True, encoding="utf8"):
        # Gebruik constructor van de basis class
        super().__init__(header, encoding)
        self.column_widths = column_widths
        
    # Aangepaste verwerking voor regels
    def parse_record(self, record):
        """Splits record in velden."""
        parsed = []
        for width in self.column_widths:
            # Splits waarde af van de rest van het record
            value = self.try_numeric(record[0:width].strip())
            parsed.append(value)
            record = record[width:]
        
        return parsed

In [ ]:
parser = FixedParser([20, 20, 8])
parser.parse("dummy_data/fixed_width_data.txt")